In [1]:
import exchange_calendars as ecals
import datetime
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow import keras
import os
from sklearn.model_selection import train_test_split
from collections import Counter

## 1. 뉴스 데이터 불러오기

In [118]:
newsdata = pd.read_csv('../data/newsfile_nouns.csv')
newsdata.head(3)

,Unnamed: 0,date,title,nouns
0,19650,20151019,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발
1,3420,20151019,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래
2,1798,20151019,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상


In [119]:
newsdata.drop(columns=['Unnamed: 0'], inplace=True)
newsdata.head(3)

,date,title,nouns
0,20151019,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발
1,20151019,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래
2,20151019,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상


In [120]:
# 데이터 형식 확인
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  int64 
 1   title   58442 non-null  object
 2   nouns   58442 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [121]:
# int > string으로 변경
newsdata = newsdata.astype({'date':'string'})
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    58442 non-null  string
 1   title   58442 non-null  object
 2   nouns   58442 non-null  object
dtypes: object(2), string(1)
memory usage: 1.3+ MB


In [122]:
# datetime 형식으로 변경
newsdata['date'] = pd.to_datetime(newsdata['date'])
newsdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58442 entries, 0 to 58441
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    58442 non-null  datetime64[ns]
 1   title   58442 non-null  object        
 2   nouns   58442 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 1.3+ MB


In [123]:
# 뉴스데이터의 처음 날짜와 마지막 날짜 확인
first_date = newsdata['date'][0]
last_date = newsdata['date'][-1:]
print(first_date, last_date)

2015-10-19 00:00:00 58441   2022-03-16
Name: date, dtype: datetime64[ns]


In [146]:
newsdata_all = newsdata[newsdata['date'].between('2015-10-19', '2022-03-16')]
newsdata_all

,date,title,nouns
0,2015-10-19,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발
1,2015-10-19,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래
2,2015-10-19,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상
3,2015-10-19,제주반도체 해외투자유치설 조회공시 요구,제주 반도체 해외 투자 유치 설 조회 공시 요구
4,2015-10-19,반도체사 인수합병 봇물올 들어 억달러 돌파,반도 체사 인수합병 봇 물올 달러 돌파
...,...,...,...
58437,2022-03-16,장덕현 삼성전기 사장 반도체 기판 위에 모든 시스템 통합하는 SoS 시장 이,장덕 현 삼성 전기 사장 반도체 판 위 모든 시스템 통합 시장 이
58438,2022-03-16,글로벌 비즈 인텔 년간 유럽 반도체 생산연구개발에 조 원 투자,글로벌 비즈 인텔 년 유럽 반도체 생산 연구개발 조 원 투자
58439,2022-03-16,미반도체지수 급등에도삼성전자 만원선에서 공방,미반 도체 지수 급등 전자 원선 방
58440,2022-03-16,SK실트론 인치 반도체 웨이퍼 공장 증설에 조 투자,실 트론 인치 반도체 웨이퍼 공장 증설 조 투자


In [135]:
news = newsdata_all['date'].to_list()
news[:5]

[Timestamp('2015-10-19 00:00:00'),
 Timestamp('2015-10-19 00:00:00'),
 Timestamp('2015-10-19 00:00:00'),
 Timestamp('2015-10-19 00:00:00'),
 Timestamp('2015-10-19 00:00:00')]

## 2. 한국 거래소 개장일 확인

In [74]:
# 한국코드
XKRX = ecals.get_calendar("XKRX")       
# 2021-01-01은 개장일인지 확인
print("2021-01-01 개장 확인: ", XKRX.is_session('2021-01-01'))    
# 오늘(22-03-19)은 개장일인지 확인
print("오늘날짜 개장일 확인: ", XKRX.is_session(datetime.date.today().strftime('%Y-%m-%d')))  
# 다음 개장일은 언제인지 확인
print("다음 개장일 날짜 확인: ", XKRX.next_open(pd.Timestamp.today().strftime('%Y-%m-%d')))

2021-01-01 개장 확인:  False
오늘날짜 개장일 확인:  False
다음 개장일 날짜 확인:  2022-03-21 00:00:00+00:00


In [87]:
XKRX.next_open(news_date[0])

Timestamp('2015-10-20 00:00:00+0000', tz='UTC')

In [128]:
# 뉴스데이터 날짜 다음날에 해당하는 개장일 날짜 확인
next_open_list = []

for day in news:
    next_open_date = XKRX.next_open(day)
    next_open_list.append(next_open_date)

In [134]:
next_open_list[:5]

[Timestamp('2015-10-20 00:00:00+0000', tz='UTC'),
 Timestamp('2015-10-20 00:00:00+0000', tz='UTC'),
 Timestamp('2015-10-20 00:00:00+0000', tz='UTC'),
 Timestamp('2015-10-20 00:00:00+0000', tz='UTC'),
 Timestamp('2015-10-20 00:00:00+0000', tz='UTC')]

In [158]:
# 뉴스나온 날짜 다음 개장일 컬럼 추가
newsdata_all['NextOpenDate'] = next_open_list
newsdata_all

,date,title,nouns,NextOpenDate
0,2015-10-19,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발,2015-10-20 00:00:00+00:00
1,2015-10-19,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래,2015-10-20 00:00:00+00:00
2,2015-10-19,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상,2015-10-20 00:00:00+00:00
3,2015-10-19,제주반도체 해외투자유치설 조회공시 요구,제주 반도체 해외 투자 유치 설 조회 공시 요구,2015-10-20 00:00:00+00:00
4,2015-10-19,반도체사 인수합병 봇물올 들어 억달러 돌파,반도 체사 인수합병 봇 물올 달러 돌파,2015-10-20 00:00:00+00:00
...,...,...,...,...
58437,2022-03-16,장덕현 삼성전기 사장 반도체 기판 위에 모든 시스템 통합하는 SoS 시장 이,장덕 현 삼성 전기 사장 반도체 판 위 모든 시스템 통합 시장 이,2022-03-17 00:00:00+00:00
58438,2022-03-16,글로벌 비즈 인텔 년간 유럽 반도체 생산연구개발에 조 원 투자,글로벌 비즈 인텔 년 유럽 반도체 생산 연구개발 조 원 투자,2022-03-17 00:00:00+00:00
58439,2022-03-16,미반도체지수 급등에도삼성전자 만원선에서 공방,미반 도체 지수 급등 전자 원선 방,2022-03-17 00:00:00+00:00
58440,2022-03-16,SK실트론 인치 반도체 웨이퍼 공장 증설에 조 투자,실 트론 인치 반도체 웨이퍼 공장 증설 조 투자,2022-03-17 00:00:00+00:00


In [159]:
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   date          58442 non-null  datetime64[ns]     
 1   title         58442 non-null  object             
 2   nouns         58442 non-null  object             
 3   NextOpenDate  58442 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), object(2)
memory usage: 4.2+ MB


In [160]:
newsdata_all = newsdata_all.astype({'NextOpenDate':'string'})
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          58442 non-null  datetime64[ns]
 1   title         58442 non-null  object        
 2   nouns         58442 non-null  object        
 3   NextOpenDate  58442 non-null  string        
dtypes: datetime64[ns](1), object(2), string(1)
memory usage: 4.2+ MB


In [161]:
a = newsdata_all['NextOpenDate'][0]
b = a.split(' ')[0]
b

'2015-10-20'

In [162]:
newsdata_all['NextOpenDate'] = newsdata_all['NextOpenDate'].apply(lambda x: x.split(' ')[0])
newsdata_all

,date,title,nouns,NextOpenDate
0,2015-10-19,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발,2015-10-20
1,2015-10-19,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래,2015-10-20
2,2015-10-19,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상,2015-10-20
3,2015-10-19,제주반도체 해외투자유치설 조회공시 요구,제주 반도체 해외 투자 유치 설 조회 공시 요구,2015-10-20
4,2015-10-19,반도체사 인수합병 봇물올 들어 억달러 돌파,반도 체사 인수합병 봇 물올 달러 돌파,2015-10-20
...,...,...,...,...
58437,2022-03-16,장덕현 삼성전기 사장 반도체 기판 위에 모든 시스템 통합하는 SoS 시장 이,장덕 현 삼성 전기 사장 반도체 판 위 모든 시스템 통합 시장 이,2022-03-17
58438,2022-03-16,글로벌 비즈 인텔 년간 유럽 반도체 생산연구개발에 조 원 투자,글로벌 비즈 인텔 년 유럽 반도체 생산 연구개발 조 원 투자,2022-03-17
58439,2022-03-16,미반도체지수 급등에도삼성전자 만원선에서 공방,미반 도체 지수 급등 전자 원선 방,2022-03-17
58440,2022-03-16,SK실트론 인치 반도체 웨이퍼 공장 증설에 조 투자,실 트론 인치 반도체 웨이퍼 공장 증설 조 투자,2022-03-17


In [163]:
newsdata_all['NextOpenDate'] = pd.to_datetime(newsdata_all['NextOpenDate'])
newsdata_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58442 entries, 0 to 58441
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          58442 non-null  datetime64[ns]
 1   title         58442 non-null  object        
 2   nouns         58442 non-null  object        
 3   NextOpenDate  58442 non-null  datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.2+ MB


In [167]:
# col의 인덱스 변경
col = newsdata_all.columns.to_numpy()
col

array(['date', 'title', 'nouns', 'NextOpenDate'], dtype=object)

In [168]:
col = col[[0,3,1,2]]
col

array(['date', 'NextOpenDate', 'title', 'nouns'], dtype=object)

In [169]:
newsdata_all = newsdata_all[col]
newsdata_all.head(3)

,date,NextOpenDate,title,nouns
0,2015-10-19,2015-10-20,먹거나 먹히거나반도체업계 합종연횡 활발,반도체 업계 합종연횡 활발
1,2015-10-19,2015-10-20,반도체 반도체 투자 감소 예상 미래,반도체 반도체 투자 감소 예상 미래
2,2015-10-19,2015-10-20,특징주제주반도체 미중 대규모 투자유치에 상,특징 반도체 중 대규모 투자 유치 상
